In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install required libraries (Colab, run just once)
!pip install -q --upgrade transformers peft trl bitsandbytes accelerate datasets

# Import necessary modules
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset

# Lightweight LLM (Mistral-7B-Instruct quantized)
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)
from huggingface_hub import login
login(token="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxx")  # Replace with your actual token


# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
)

# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)

# Load and prepare the dataset
dataset = load_dataset("mlabonne/guanaco-llama2-1k")

def rename_text_to_prompt(example):
    example["prompt"] = example["text"]
    return example

train_dataset = dataset["train"].map(rename_text_to_prompt)

# Define reward functions
def reward_len(prompts, completions, completion_ids, **kwargs):
    scores = [len(comp) / 100.0 for comp in completions]
    return torch.tensor(scores, dtype=torch.float32)

def reward_token_diversity(prompts, completions, completion_ids, **kwargs):
    scores = [len(set(comp.split())) / 50.0 for comp in completions]
    return torch.tensor(scores, dtype=torch.float32)

def reward_reasoning(prompts, completions, completion_ids, **kwargs):
    keywords = ["because", "therefore", "thus", "hence", "consequently", "as a result"]
    scores = []
    for comp in completions:
        score = sum(1 for word in keywords if word in comp.lower())
        scores.append(score)
    return torch.tensor(scores, dtype=torch.float32)

# GRPO training configuration
training_args = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    num_generations=2,
    max_prompt_length=256,
    max_completion_length=64,
    max_steps=25,
    save_steps=25,
    max_grad_norm=0.1,
    report_to="none",
    output_dir="outputs",
)

# Initialize trainer
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[reward_len, reward_token_diversity, reward_reasoning],
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Evaluate after training
eval_prompts = [
    "What is the value of 2 + 2? Just give the number.",
    "Explain why the sky is blue.",
    "If it's raining, I will take an umbrella. It's not raining. Will I take an umbrella? Why?"
]
model.eval()
for prompt in eval_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    print(f"\nPrompt: {prompt}")
    print(f"Response: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,-0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le


Prompt: What is the value of 2 + 2? Just give the number.
Response: What is the value of 2 + 2? Just give the number.

The value of 2 + 2 is 4.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Prompt: Explain why the sky is blue.
Response: Explain why the sky is blue.

The sky appears blue because of a process called Rayleigh scattering. When sunlight reaches Earth, it is made up of different colors, each with a different wavelength. Blue light has a shorter wavelength and is scattered in all directions more easily than other colors like red, yellow, or green.

As sunlight travels through Earth's atmosphere, the shorter blue light waves collide with molecules in the air more frequently than the longer wavelengths of other colors. When these

Prompt: If it's raining, I will take an umbrella. It's not raining. Will I take an umbrella? Why?
Response: If it's raining, I will take an umbrella. It's not raining. Will I take an umbrella? Why?

If it's raining, an umbrella would be useful to protect you from the rain. Since it's not raining, there's no need for an umbrella. Therefore, you wouldn't take an umbrella. However, if you have a habit of always carrying an umbrella, you mi

## Save and Load the model

In [ ]:

import os
drive_output_folder = "/content/drive/MyDrive/MyFinetunedModels"
model_save_path = os.path.join(drive_output_folder, "my_grpo_finetuned_mistral_model")

# Create the directory in Google Drive if it doesn't exist
os.makedirs(model_save_path, exist_ok=True)

trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path) # Also save the tokenizer with the model

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to /content/drive/MyDrive/MyFinetunedModels/my_grpo_finetuned_mistral_model


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig # Import PeftModel and PeftConfig for loading

# Define the directory where your model was saved
saved_model_path = "./my_grpo_finetuned_model"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(saved_model_path)

# Load the PEFT configuration
peft_config = PeftConfig.from_pretrained(saved_model_path)

# Load the base model using the original model_name and quantization config
# You need the same quantization config as during training if you want to load it quantized.
model_name_original = "mistralai/Mistral-7B-Instruct-v0.3" # Original base model name
quant_config_load = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name_original,
    quantization_config=quant_config_load,
    device_map="auto"
)

# Load the PEFT adapters and attach them to the base model
model = PeftModel.from_pretrained(base_model, saved_model_path)

model.eval() # Set the model to evaluation mode

# Example inference with the loaded model
prompt = "Tell me a short story about a brave knight."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(f"\nPrompt: {prompt}")
print(f"Response: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")